In [1]:
# -*- coding: utf-8 -*-
"""
Code for generating the first data figure in the manuscript.

Authors: Julia Sprenger, Lyuba Zehl, Michael Denker


Copyright (c) 2017, Institute of Neuroscience and Medicine (INM-6),
Forschungszentrum Juelich, Germany
All rights reserved.

Redistribution and use in source and binary forms, with or without
modification, are permitted provided that the following conditions are met:

* Redistributions of source code must retain the above copyright notice, this
list of conditions and the following disclaimer.
* Redistributions in binary form must reproduce the above copyright notice,
this list of conditions and the following disclaimer in the documentation
and/or other materials provided with the distribution.
* Neither the names of the copyright holders nor the names of the contributors
may be used to endorse or promote products derived from this software without
specific prior written permission.

THIS SOFTWARE IS PROVIDED BY THE COPYRIGHT HOLDERS AND CONTRIBUTORS "AS IS" AND
ANY EXPRESS OR IMPLIED WARRANTIES, INCLUDING, BUT NOT LIMITED TO, THE IMPLIED
WARRANTIES OF MERCHANTABILITY AND FITNESS FOR A PARTICULAR PURPOSE ARE
DISCLAIMED. IN NO EVENT SHALL THE COPYRIGHT HOLDER OR CONTRIBUTORS BE LIABLE
FOR ANY DIRECT, INDIRECT, INCIDENTAL, SPECIAL, EXEMPLARY, OR CONSEQUENTIAL
DAMAGES (INCLUDING, BUT NOT LIMITED TO, PROCUREMENT OF SUBSTITUTE GOODS OR
SERVICES; LOSS OF USE, DATA, OR PROFITS; OR BUSINESS INTERRUPTION) HOWEVER
CAUSED AND ON ANY THEORY OF LIABILITY, WHETHER IN CONTRACT, STRICT LIABILITY,
OR TORT (INCLUDING NEGLIGENCE OR OTHERWISE) ARISING IN ANY WAY OUT OF THE USE
OF THIS SOFTWARE, EVEN IF ADVISED OF THE POSSIBILITY OF SUCH DAMAGE.
"""

'\nCode for generating the first data figure in the manuscript.\n\nAuthors: Julia Sprenger, Lyuba Zehl, Michael Denker\n\n\nCopyright (c) 2017, Institute of Neuroscience and Medicine (INM-6),\nForschungszentrum Juelich, Germany\nAll rights reserved.\n\nRedistribution and use in source and binary forms, with or without\nmodification, are permitted provided that the following conditions are met:\n\n* Redistributions of source code must retain the above copyright notice, this\nlist of conditions and the following disclaimer.\n* Redistributions in binary form must reproduce the above copyright notice,\nthis list of conditions and the following disclaimer in the documentation\nand/or other materials provided with the distribution.\n* Neither the names of the copyright holders nor the names of the contributors\nmay be used to endorse or promote products derived from this software without\nspecific prior written permission.\n\nTHIS SOFTWARE IS PROVIDED BY THE COPYRIGHT HOLDERS AND CONTRIBUTOR

In [2]:
import os

import csv

import numpy as np
from scipy import stats
import quantities as pq
import matplotlib.pyplot as plt

from matplotlib import gridspec, ticker

from reachgraspio import reachgraspio

import odml.tools

from neo import utils as neo_utils
from neo_utils import load_segment
import odml_utils

# Imports from example_nix.py
from neo import Block, Segment
from neo.io import NixIO
from neo.utils import cut_segment_by_epoch, add_epoch, get_events

In [3]:
# =============================================================================
# Define data and metadata directories
# =============================================================================


def get_monkey_datafile(monkey):
    if monkey == "Lilou":
        return "l101210-001"  # ns2 (behavior) and ns5 present
    elif monkey == "Nikos2":
        return "i140703-001"  # ns2 and ns6 present
    else:
        return ""

# Enter your dataset directory here
datasetdir = "../multielectrode_grasp/datasets_blackrock/"

trialtype_colors = {
    'SGHF': 'MediumBlue', 'SGLF': 'Turquoise',
    'PGHF': 'DarkGreen', 'PGLF': 'YellowGreen',
    'LFSG': 'Orange', 'LFPG': 'Yellow',
    'HFSG': 'DarkRed', 'HFPG': 'OrangeRed',
    'SGSG': 'SteelBlue', 'PGPG': 'LimeGreen',
    'NONE': 'k', 'PG': 'k', 'SG': 'k', 'LF': 'k', 'HF': 'k'}

event_colors = {
    'TS-ON': 'Gray',  # 'TS-OFF': 'Gray',
    'WS-ON': 'Gray',  # 'WS-OFF': 'Gray',
    'CUE-ON': 'Gray',
    'CUE-OFF': 'Gray',
    'GO-ON': 'Gray',  # 'GO-OFF': 'Gray',
    #    'GO/RW-OFF': 'Gray',
    'SR': 'Gray',  # 'SR-REP': 'Gray',
    'RW-ON': 'Gray',  # 'RW-OFF': 'Gray',
    'STOP': 'Gray'}



In [4]:
# =============================================================================
# Plot helper functions
# =============================================================================


def force_aspect(ax, aspect=1):
    ax.set_aspect(abs(
        (ax.get_xlim()[1] - ax.get_xlim()[0]) /
        (ax.get_ylim()[1] - ax.get_ylim()[0])) / aspect)


def get_arraygrid(signals, chosen_el):
    array_grid = np.ones((10, 10)) * 0.7

    rejections = np.logical_or(signals.array_annotations['electrode_reject_HFC'],
                               signals.array_annotations['electrode_reject_LFC'],
                               signals.array_annotations['electrode_reject_IFC'])

    for sig_idx in range(signals.shape[-1]):
        connector_aligned_id = signals.array_annotations['connector_aligned_ids'][sig_idx]
        x, y = int((connector_aligned_id -1)// 10), int((connector_aligned_id - 1) % 10)

        if np.asarray(signals.array_annotations['channel_ids'][sig_idx], dtype=int) == chosen_el:
            array_grid[x, y] = -0.7
        elif rejections[sig_idx]:
            array_grid[x, y] = -0.35
        else:
            array_grid[x, y] = 0

    return np.ma.array(array_grid, mask=np.isnan(array_grid))



In [5]:
# =============================================================================
# Load data and metadata for a monkey
# =============================================================================
# CHANGE this parameter to load data of the different monkeys
# monkey = 'Nikos2'
monkey = 'Lilou'

#chosen_el = {'Lilou': 71, 'Nikos2': 63}
#chosen_units = {'Lilou': range(1, 5), 'Nikos2': range(1, 5)}




In [6]:
# =============================================================================
# Load data
#
# As a first step, we load the data file into memory as a Neo object.
# =============================================================================

# Specify the datafile
datafile = get_monkey_datafile(monkey)

# Specify the path to the recording session to load, eg,
session_path = os.path.join("..", "multielectrode_grasp","datasets_nix", datafile + ".nix")

# Open the session for reading
session = NixIO(session_path)

# Channel ID to plot
#target_channel_id = 62


# Read the complete dataset in lazy mode generating all neo objects,
# but not loading data into memory.  The lazy neo structure will contain objects
# to capture all recorded data types (time series at 1000Hz (ns2) and 30kHz (ns6)
# scaled to units of voltage, sorted spike trains, spike waveforms and events)
# of the recording session and return it as a Neo Block. The
# time shift of the ns2 signal (LFP) induced by the online filter is
# automatically corrected for by a heuristic factor stored in the metadata
# (correct_filter_shifts=True).

block = session.read_block()

# Validate there is only a single Segment present in the block
assert len(block.segments) == 1

# loading data content of all data objects during the first 300 seconds
# data_segment = load_segment(block.segments[0], time_range=(None, 300*pq.s))
##data_segment = block.segments[0]
segment = block.segments[0]

print("Session loaded.")

Session loaded.


In [7]:
# get start and stop events of trials
start_events = neo_utils.get_events(
    segment,
    **{
    'name': 'TrialEvents',
    'trial_event_labels': 'TS-ON',
    'performance_in_trial': 255})
stop_events = neo_utils.get_events(
    segment,
    **{
    'name': 'TrialEvents',
    'trial_event_labels': 'STOP',
    'performance_in_trial': 255})

# there should only be one event object for these conditions
assert len(start_events) == 1
assert len(stop_events) == 1


In [8]:
# insert epochs between 10ms before TS to 50ms after RW corresponding to trails
ep = neo_utils.add_epoch(
    segment,
    start_events[0],
    stop_events[0],
    pre=0 * pq.ms, #-250 * pq.ms,
    post=0 * pq.ms, #500 * pq.ms,
    trial_status='complete_trials')
ep.array_annotate(trial_type=start_events[0].array_annotations['belongs_to_trialtype'],
                  trial_performance=start_events[0].array_annotations['performance_in_trial'])


epochs = neo_utils.get_epochs(segment, **{'trial_status': 'complete_trials'})
assert len(epochs) == 1


"\nepochs_rest = neo_utils.get_epochs(segment, **{'label': 'rest'})\nassert len(epochs_rest) == 1\n\nepochs_mvt = neo_utils.get_epochs(segment, **{'label': 'mvt'})\nassert len(epochs_mvt) == 1\n"

In [9]:
# use most raw neuronal data if multiple versions are present
max_sampling_rate = max([a.sampling_rate for a in segment.analogsignals])
idx = 0
while idx < len(segment.analogsignals):
    signal = segment.analogsignals[idx]
    if signal.annotations['neural_signal'] and signal.sampling_rate < max_sampling_rate:
        segment.analogsignals.pop(idx)
    else:
        idx += 1

# cut segments according to inserted 'complete_trials' epochs and reset trial times
cut_segments = neo_utils.cut_segment_by_epoch(segment, epochs[0], reset_time=True)


In [58]:
# =============================================================================
# Feature and label extractions from data
# =============================================================================

# Determine a time windows to slide on the data
window = 500 # ms (Choose here the size of the sliding window)
window_len = window/1000*30000 # Number of points @ 30kHz
window_time = (window * pq.ms).rescale(pq.s) # Window size in second

# Passing through all segments with the time window, at each time frame, extract 
# the feature (1 value) and the label (rest or mvt). Write values in a csv file.

electrode = 4 #comme figure 6

# Feature 1 : mean over all electodes
#feature = 'mean_over_electrodes'
feature1 = 'mean_seg'
feature2 = 'max-min'

# Open a csv file in exclusive creating mode
with open('features/'+feature1+'_'+feature2+'el_'+str(electrode)+'_11juin_v1.csv', mode='x') as fobj:
    writer = csv.writer(fobj)
    writer.writerow(['id_segment', 'label', feature1, feature2]) 
    for index, seg in enumerate(cut_segments) :
        
        # Calculate the number of resting frames in the segment
        for events in seg.events:
            if 'GO-ON' in events.labels :
                start_arg = np.where(events.labels=='TS-ON')[0][0]
                start_time = events.times[start_arg]
                go_arg = np.where(events.labels=='GO-ON')[0][0]
                go_time = events.times[go_arg]
                stop_arg = np.where(events.labels=='STOP')[0][0]
                stop_time = events.times[stop_arg]
        frames_rest = round((go_time - start_time)/window_time).item()    

        # Extract raw data of the segment
        raw_data = [a for a in seg.analogsignals if a.annotations['neural_signal']]
        assert len(raw_data) == 1
        raw_data = raw_data[0]
        raw_signal = raw_data[:, electrode]

        # Slide the window throught the data of the segment to extract features and labels
        for frame in range(int(np.floor(len(raw_signal)/window_len))) :
            data_window = raw_signal[int(np.floor(frame*window_len)):int(np.floor((frame+1)*window_len))]
            
            # Extract feature for the window
            mean_seg = np.mean(data_window)
            max_min = max(data_window) - min(data_window)
                
            # Extract label
            if frame<frames_rest:
                label = 'rest'
            else:
                label = 'mvt'

            # Write in csv file
            writer.writerow([index, label, mean_seg, max_min]) 

